In [2]:
import pandas as pd

#Loading the original tables into dataframes
#df_accident=pd.read_csv('~/code/MagicaBleps/UK-road-safety/raw_data/dft-road-casualty-statistics-accident-1979-2021.csv', sep=',', header=0)
#df_vehicle=pd.read_csv('~/code/MagicaBleps/UK-road-safety/raw_data/dft-road-casualty-statistics-vehicle-1979-2021.csv', sep=',', header=0)
#df_casualty=pd.read_csv('~/code/MagicaBleps/UK-road-safety/raw_data/dft-road-casualty-statistics-casualty-1979-2021.csv', sep=',', header=0)

In [3]:
import pygeohash as gh

In [2]:
#Filtering the dataframes to get dta from 1999 only
#df_accident_1999=df_accident[df_accident['accident_year']>=1999]
#df_vehicle_1999=df_vehicle[df_vehicle['accident_year']>=1999]
#df_casualty_1999=df_casualty[df_casualty['accident_year']>=1999]

In [3]:
#Exporting the data from 1999 as csv files --> the new raw data files
#df_accident_1999.to_csv('~/code/MagicaBleps/UK-road-safety/raw_data/dft-road-casualty-statistics-accident-1999-2021.csv')
#df_vehicle_1999.to_csv('~/code/MagicaBleps/UK-road-safety/raw_data/dft-road-casualty-statistics-vehicle-1999-2021.csv')
#df_casualty_1999.to_csv('~/code/MagicaBleps/UK-road-safety/raw_data/dft-road-casualty-statistics-casualty-1999-2021.csv')

In [4]:
df_accidents=pd.read_csv('~/code/MagicaBleps/UK-road-safety/raw_data/dft-road-casualty-statistics-accident-1999-2021.csv', index_col=0)
df_accidents.head()

/tmp/ipykernel_31702/1322138315.py:1: DtypeWarning: Columns (1,3,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df_accidents=pd.read_csv('~/code/MagicaBleps/UK-road-safety/raw_data/dft-road-casualty-statistics-accident-1999-2021.csv', index_col=0)


,accident_index,accident_year,accident_reference,location_easting_osgr,location_northing_osgr,longitude,latitude,police_force,accident_severity,number_of_vehicles,...,pedestrian_crossing_physical_facilities,light_conditions,weather_conditions,road_surface_conditions,special_conditions_at_site,carriageway_hazards,urban_or_rural_area,did_police_officer_attend_scene_of_accident,trunk_road_flag,lsoa_of_accident_location
4883216,1999010SU0945,1999,010SU0945,519490.0,203300.0,-0.271752,51.715661,1,3,1,...,0,1,4,1,0,0,2,1,1,E01023583
4883217,1999010SU0946,1999,010SU0946,521740.0,201070.0,-0.239977,51.695136,1,3,2,...,0,6,4,1,0,0,2,1,1,E01023584
4883218,1999010SU0947,1999,010SU0947,519610.0,203240.0,-0.270037,51.715096,1,3,2,...,0,4,1,2,0,0,2,1,1,E01023583
4883219,1999010SU0948,1999,010SU0948,520090.0,202830.0,-0.263233,51.711309,1,2,2,...,0,6,1,1,0,0,2,1,1,E01023583
4883220,1999010SU0949,1999,010SU0949,522640.0,200320.0,-0.227225,51.688200,1,3,4,...,0,1,1,2,0,0,2,1,1,E01023584


In [57]:
#Creating a small dataset to be loaded quickly
df_accidents_small=df_accidents[df_accidents['accident_year']==2019]
df_accidents_small.to_csv('~/code/MagicaBleps/UK-road-safety/raw_data/dft-road-casualty-statistics-accident-small.csv')

In [7]:
def add_time_columns(df):
    df["date"] = pd.to_datetime(df["date"],format='%d/%m/%Y')
    df["IsWeekend"] = df['date'].dt.weekday > 5
    df['month'] = df['date'].dt.month
    df['Daytime'] = df['time'].apply(lambda x: "morning rush (5-10)" if '05:00' < x <= '10:00'
                                   else ("office hours (10-15)" if '10:00' < x <= '15:00'
                                         else( "afternoon rush (15-19)" if '15:00' < x <= '19:00'
                                              else ("evening (19-23)" if '19:00' < x <= '23:00'
                                                    else("night (23-5")))))
    return df


In [8]:
def add_geohash(df,p=5):
    df['geohash']=df.apply(lambda x: gh.encode(x.latitude, x.longitude, precision=p), axis=1)
    return df


In [9]:
def delete_columns(df):
    columns_to_delete=['accident_reference', 'accident_index',
                       'location_easting_osgr','location_northing_osgr',
                       'police_force','local_authority_district',
                       'local_authority_ons_district','local_authority_highway',
                       'first_road_class', 'first_road_number',
                       'road_type', 'speed_limit',
                       'junction_detail', 'junction_control',
                       'pedestrian_crossing_human_control',
                       'pedestrian_crossing_physical_facilities',
                       'light_conditions', 'weather_conditions',
                       'road_surface_conditions','special_conditions_at_site',
                       'second_road_class', 'second_road_number',
                       'carriageway_hazards', 'urban_or_rural_area',
                       'did_police_officer_attend_scene_of_accident',
                       'trunk_road_flag','lsoa_of_accident_location'
                       ]
    df_new=df.drop(columns_to_delete,axis=1)
    return df_new

def fix_missing_values(df):
    #dropping rows without lat and lon
    df_new=df.dropna(axis=0,subset=['longitude','latitude'])

    return df_new

def prepare_data_for_groupby(df,precision):
    '''The function takes the list of accidents in time and prepares it for the groupby step.
    The precision parameter is used for the geohash step.'''
    df_new=delete_columns(df)
    df_new=fix_missing_values(df_new)
    df_new=add_geohash(df_new,p=precision)
    df_new=add_time_columns(df_new)
    
    return df_new

In [10]:
def data_for_analysis(df,granularity='M'):
    '''granularity M: Monthly  W: Weekly 
       Returns a dataframe (date, No. of Accidents)''' 
    if granularity=='W':
        g='W-MON'
    else:
        g='M'
        
    test = pd.DataFrame(df.set_index('date').resample(g).size())
    test.columns = ['Accidents']
    return test

In [11]:
test=delete_columns(df_accidents)
test.head()

,accident_year,longitude,latitude,accident_severity,number_of_vehicles,number_of_casualties,date,day_of_week,time
4883216,1999,-0.271752,51.715661,3,1,1,25/12/1999,7,09:30
4883217,1999,-0.239977,51.695136,3,2,1,17/12/1999,6,18:38
4883218,1999,-0.270037,51.715096,3,2,2,15/12/1999,4,18:04
4883219,1999,-0.263233,51.711309,2,2,1,02/12/1999,5,04:10
4883220,1999,-0.227225,51.688200,3,4,3,04/12/1999,7,09:51


In [12]:
test=test.dropna(axis=0,subset=['longitude','latitude'])
test.head()

,accident_year,longitude,latitude,accident_severity,number_of_vehicles,number_of_casualties,date,day_of_week,time
4883216,1999,-0.271752,51.715661,3,1,1,25/12/1999,7,09:30
4883217,1999,-0.239977,51.695136,3,2,1,17/12/1999,6,18:38
4883218,1999,-0.270037,51.715096,3,2,2,15/12/1999,4,18:04
4883219,1999,-0.263233,51.711309,2,2,1,02/12/1999,5,04:10
4883220,1999,-0.227225,51.688200,3,4,3,04/12/1999,7,09:51


In [13]:
test=add_time_columns(test)
test.head()

,accident_year,longitude,latitude,accident_severity,number_of_vehicles,number_of_casualties,date,day_of_week,time,IsWeekend,month,Daytime
4883216,1999,-0.271752,51.715661,3,1,1,1999-12-25,7,09:30,False,12,morning rush (5-10)
4883217,1999,-0.239977,51.695136,3,2,1,1999-12-17,6,18:38,False,12,afternoon rush (15-19)
4883218,1999,-0.270037,51.715096,3,2,2,1999-12-15,4,18:04,False,12,afternoon rush (15-19)
4883219,1999,-0.263233,51.711309,2,2,1,1999-12-02,5,04:10,False,12,night (23-5
4883220,1999,-0.227225,51.688200,3,4,3,1999-12-04,7,09:51,False,12,morning rush (5-10)


In [14]:
test_data=data_for_analysis(test,granularity='W')
test_data.head(52)

,Accidents
date,
1999-01-04,1586
1999-01-11,4623
1999-01-18,5000
1999-01-25,4589
1999-02-01,4137
1999-02-08,3986
1999-02-15,4320
1999-02-22,3902
1999-03-01,4321


In [34]:
test_data.shape

(1201, 1)

In [15]:
test_hashed=add_geohash(test,5)

In [29]:
import matplotlib.pyplot as plt
import seaborn as sns

hashes=pd.DataFrame(test_hashed[['geohash','accident_year']].groupby('geohash').count())
hashes.columns=['Accidents']
hashes.sort_values('Accidents',inplace=True,ascending=False)
hashes.shape


(14683, 1)

In [31]:
hashes.describe()

,Accidents
count,14683.000000
mean,259.925901
std,883.711740
min,1.000000
25%,11.000000
50%,49.000000
75%,159.500000
max,28379.000000


In [56]:
hashes[hashes['Accidents']>=100]

,Accidents
geohash,
gcpvj,28379
gcpvh,26311
gcpuv,24296
gcpvn,23827
gcpuu,18940
...,...
gcmbg,100
gcqsb,100
gcpx0,100


In [33]:
test_hashed_6=add_geohash(test,6)

In [39]:
hashes_6=pd.DataFrame(test_hashed_6[['geohash','accident_year']].groupby('geohash').count())
hashes_6.columns=['Accidents']
hashes_6.sort_values('Accidents',inplace=True,ascending=False)
hashes_6.shape

(184578, 1)

In [49]:
hashes_6[hashes_6['Accidents']>=100]

,Accidents
geohash,
gcpvj0,2045
gcpvhc,1959
gcpvj4,1871
gcpvj1,1816
gcpuv2,1744
...,...
gcwzyg,100
gcx6bn,100
u120gg,100


In [55]:
data_for_analysis(test_hashed_6[test_hashed_6['geohash']=='gcpvj0'],'W').head(50)

,Accidents
date,
1999-01-04,1
1999-01-11,1
1999-01-18,1
1999-01-25,1
1999-02-01,1
1999-02-08,4
1999-02-15,3
1999-02-22,5
1999-03-01,2
